In [4]:
# if german spacy not present uncomment this:
#!python3 -m spacy download de

In [1]:
import torch
import numpy  as np
from torch import nn, optim
import time
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
spacy_english = spacy.load('en')
spacy_german = spacy.load('de')

In [2]:
# tokenize languages
def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

# reversing order has been shown to imporve model!!! 
def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)][::-1] 



In [3]:
SOURCE = Field(tokenize = tokenize_english, 
               init_token = "<sos>",
               eos_token = "<eso>",
               lower = True
              )

TARGET = Field(tokenize = tokenize_german, 
               init_token = "<sos>",
               eos_token = "<eso>",
               lower = True
              )

train_data, valid_data, test_data = Multi30k.splits(exts = ('.en', '.de'),
                                                    fields= (SOURCE, TARGET))


/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [4]:
print(train_data.examples[0].src)
print(train_data.examples[0].trg)

['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']
['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']


In [8]:
len(train_data.examples), len(test_data.examples), len(test_data.examples)

(29000, 1000, 1000)

In [13]:
SOURCE.build_vocab(train_data, min_freq = 2)
TARGET.build_vocab(train_data, min_freq = 2)
len(SOURCE.vocab), len(TARGET.vocab)

(5893, 7854)